In [1]:
import numpy as np
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy import optimize

In this model project we want to solve the Solow model with humancapital. We take the model from 'Introducing Advanced Macroeconomics - Growth and Business Cycles'. 3 edition 2022, Sørensen, Peter Birch and Whitta-Jacobsen, Hans Jørgen.

The Solow model with human capital from Mankiw, Weil and Romer is an extenstion of the general Solow model, that helps explain why international investments does not flow to poorer countries as predicted by the general Solow model. The modification finds itself in the addition of human capital, which is mostly measured as years of education and experience in general, and sheds greater light on why some countries are rich and some are poor. 

# The model

The Solow model with human capital is given with these equations. 1) stating that output is given as a Cobb-Douglas production function. 2 and 3) stating that the real rental rate on capital and real wage is given from the marginalproduct of capital and labour respectively. 4 and 5) stating the accumulation equation of capital and human capital. 6) stating that labour grows with the exogeneously rate of n. 7) stating that technology grows with the exogeneously rate of g.

\begin{align}
Y_t & = K_t^{\alpha} H_t ^\phi (A_t L_t)^{1-\alpha-\phi}, &0<\alpha,\phi<1, \ \alpha+\phi < 1 \\
r_t & = \alpha (\frac{K_t} {A_t L_t})^{\alpha-1} (\frac{H_t} {A_t L_t}) ^\phi \\
w_t & = (1-\alpha) (\frac{K_t} {A_t L_t})^\alpha (\frac{H_t} {A_t L_t})^\phi A_t \\
K_{t+1} & = s_K Y_t + (1-\delta)K_t, &0<\delta<1 \\
H_{t+1} & = s_H Y_t + (1-\delta)K_t, \\
L_{t+1} & = (1+n) L_t, &-1<n<1 \\
A_{t+1} & = (1+g) A_t, \\
\end{align}
where $s_h + s_k < 1$.

# Solving the model analytically (symbolicaly) 

In [14]:
#First we define the numerous symbols with sympy
Y = sm.symbols('Y_t')         #output in the economy
y = sm.symbols('y_t')         #Output per worker
K = sm.symbols('K_t')         #Capital
A = sm.symbols('A_t')         #Technology
H = sm.symbols('H_t')         #Human capital
k = sm.symbols('k_t')         #Capital per worker
r = sm.symbols('r_t')         #Real rental rate
w = sm.symbols('w_t')         #Real wage
s = sm.symbols('s')           #Rate of savings
n = sm.symbols('n')           #Population growth
g = sm.symbols('g')           #Growth in technology
L = sm.symbols('L_t')         #Labour
alpha = sm.symbols('alpha')   #Income distribution to capital and labour
phi = sm.symbols('phi')       #Income distribution to human capital
delta = sm.symbols('delta')   #Depreciation rate on (human)capital
kstar = sm.symbols('k_t^*')   #Steady state for capital
hstar = sm.symbols('h_t^*')   #Steady state for human capital
ystar = sm.symbols('y^*')     #Steady state for output per worker

The Solow equations for human capital and capital is given by as given from the book:

\begin{equation}
\tilde{k}_{t+1} - \tilde{k}_t = \frac{1}{(1+n)(1+g)}(s_K \tilde{k}_t^{\alpha} \tilde{h}_t^{\phi} - (n+g+\delta +ng)\tilde{k}_t)
\end{equation}
\begin{equation}
\tilde{h}_{t+1} - \tilde{h}_t = \frac{1}{(1+n)(1+g)}(s_H \tilde{k}_t^{\alpha} \tilde{h}_t^{\phi} - (n+g+\delta +ng)\tilde{h}_t)
\end{equation}

This equations can be turned into sympy functions and 